In [1]:
from megs.data import Gamma 

In [2]:
from megs.data.simulations import select_illustris_galaxies

In [3]:
select_illustris_galaxies(basepath="/export/data/ucakir/TNG100/TNG100-1/output/", snapshot = 99, M_min = 10**9.5, M_max = 10**13, particle_type="stars")

Found 11975 galaxies with stellar mass between 3162277660.1683793 and 10000000000000 Msun/h.


array([     0,      1,      2, ..., 727485, 738596, 752194])

In [4]:
import numpy as np 
import h5py

halo_ids = np.load("/export/home/ucakir/MEGS/MEGS/src/gamma_ids_outlier_removed.npy")
outlier_ids = np.load("/export/home/ucakir/MEGS/MEGS/src/outlier_ids.npy")

old_datapath = "/export/home/ucakir/MEGS/MEGS/src/megs/data/galaxy_data.hdf5"
new_gamma_path = "/export/data/ucakir/final_gamma.hdf5"

In [5]:
gamma_ids = np.load("/export/home/ucakir/MEGS/MEGS/src/megs/gamma_ids.npy")
print(gamma_ids.shape)

(11975,)


In [6]:
print(halo_ids.shape)

(11778,)


In [7]:
old_data = Gamma(old_datapath)

File: /export/home/ucakir/MEGS/MEGS/src/megs/data/galaxy_data.hdf5
Group: Galaxies
    Group: Attributes
        Dataset: halo_id (float32) ((12484,))
        Dataset: mass (float32) ((12484,))
    Group: Particles
        Group: stars
            Group: Images
                Group: dim2
                    Dataset: GFM_Metallicity (float32) ((12484, 64, 64))
                    Dataset: GFM_StellarFormationTime (float32) ((12484, 64, 64))
                    Dataset: Masses (float32) ((12484, 64, 64))
                Group: dim3
                    Dataset: GFM_Metallicity (float32) ((12484, 64, 64, 64))
                    Dataset: GFM_StellarFormationTime (float32) ((12484, 64, 64, 64))
                    Dataset: Masses (float32) ((12484, 64, 64, 64))


In [8]:
new_gamma_data = Gamma(new_gamma_path)

File: /export/data/ucakir/final_gamma.hdf5
Group: Galaxies
    Group: Attributes
        Dataset: halo_id (float32) ((12484,))
        Dataset: mass (float32) ((12484,))
    Group: Particles
        Group: stars
            Group: Images
                Group: dim2
                    Dataset: GFM_Metallicity (float32) ((12484, 64, 64))
                    Dataset: GFM_StellarFormationTime (float32) ((12484, 64, 64))
                    Dataset: Masses (float32) ((12484, 64, 64))
                Group: dim3
                    Dataset: GFM_Metallicity (float32) ((12484, 64, 64, 64))
                    Dataset: GFM_StellarFormationTime (float32) ((12484, 64, 64, 64))
                    Dataset: Masses (float32) ((12484, 64, 64, 64))


In [8]:
old_data = Gamma(old_datapath)
ids_in_old = old_data.get_attribute("halo_id")

mask = np.isin(ids_in_old, gamma_ids)
# filter out the outliers
mask = np.logical_and(mask, ~np.isin(ids_in_old, outlier_ids))

old_data.mask = mask
old_data.get_attribute("halo_id").shape


File: /export/home/ucakir/MEGS/MEGS/src/megs/data/galaxy_data.hdf5
Group: Galaxies
    Group: Attributes
        Dataset: halo_id (float32) ((12484,))
        Dataset: mass (float32) ((12484,))
    Group: Particles
        Group: stars
            Group: Images
                Group: dim2
                    Dataset: GFM_Metallicity (float32) ((12484, 64, 64))
                    Dataset: GFM_StellarFormationTime (float32) ((12484, 64, 64))
                    Dataset: Masses (float32) ((12484, 64, 64))
                Group: dim3
                    Dataset: GFM_Metallicity (float32) ((12484, 64, 64, 64))
                    Dataset: GFM_StellarFormationTime (float32) ((12484, 64, 64, 64))
                    Dataset: Masses (float32) ((12484, 64, 64, 64))


(11727,)

In [26]:
new_halo_ids = old_data.get_attribute("halo_id")
new_mass = old_data.get_attribute("mass")
new_metallicity_image_2d =old_data.get_image("stars", "GFM_Metallicity",dim = 2)
new_stellar_age_2d = old_data.get_image("stars", "GFM_StellarFormationTime",dim = 2)
new_masses_2d = old_data.get_image("stars", "Masses",dim = 2)

new_metallicity_image_3d =old_data.get_image("stars", "GFM_Metallicity",dim = 3)
new_stellar_age_3d = old_data.get_image("stars", "GFM_StellarFormationTime",dim = 3)
new_masses_3d = old_data.get_image("stars", "Masses",dim = 3)


In [9]:
from tqdm import tqdm
FIELDS = ["GFM_Metallicity", "GFM_StellarFormationTime", "Masses"]
attributes = ["mass", "halo_id"]

particle = "stars"
from megs.data.generate import _create_data_structure

# Create the data structure
_create_data_structure(
    n_galaxies=len(new_halo_ids),
    image_res = 64,
    galaxy_parameters=attributes,
    particle_types = "stars",
    fields = FIELDS,
    path = new_gamma_path,
)


# OPen new file
# overvrite the old file
with h5py.File(new_gamma_path,"w") as f:
    for parameter in f["Galaxies/Attributes"].keys():
        f["Galaxies/Attributes"][parameter] = old_data.get_attribute(parameter)
    
    for d in tqdm(f["Galaixes"]["Particles"][particle]["Images"].keys()):
        for field in tqdm(f["Galaxies"]["Particles"][particle]["Images"][
                        d
                    ].keys()):
                        # Get the image
                        dim = int(d[-1]) 
                        print(dim)
                        f["Galaixes"]["Particles"][particle]["Images"][d][field] = old_data.get_image(particle, field, dim = dim)
                        
    

KeyError: 'Unable to open object (component not found)'

In [11]:
with h5py.File(new_gamma_path,"r") as f:
    print(f.keys())

<KeysViewHDF5 []>


In [23]:
# Check if there are ids in ids_in_old but not in gamma_ids
# If there are, we need to remove them from the old_data
test = []
for i in ids_in_old:
    if i not in gamma_ids:
        #check if it is an outlier
        if i not in outlier_ids:
            test.append(i)

In [24]:
len(test)

547